<center>
<h1 style="color:red; font-size:70px;"> FeelNet</h1>

In [12]:
import numpy as np

from tensorflow import keras
from keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Flatten, Dense, MaxPooling2D, Conv2D, Dropout

In questo Jupiter Notebook è presente l'implementazione di una CNN il cui obiettivo è quello di determinare che emozione associare ad una particolare espressione facciale. Il dataset utilizzato per analizzare e testare le performance della rete è Fer-2013, contenente indicativamente 35000 immagini.

## Costruzione della rete

In [ ]:
# Funzione per la costruzione del modello
def buildMod(imW, imH, numcl):

    model = Sequential()
    model.add(Input(shape=(imW, imH, 3)))

    # Primo blocco convoluzionale
    model.add(Conv2D(16, 3, padding='same', activation='relu'))
    model.add(Conv2D(16, 3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(0.2))

    # Secondo blocco convoluzionale
    model.add(Conv2D(32, 3, padding='same', activation='relu'))
    model.add(Conv2D(32, 3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(0.3))

    # Terzo blocco convoluzionale
    model.add(Conv2D(64, 3, padding='same', activation='relu'))
    model.add(Conv2D(64, 3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(0.4))

    # Parte di classificazione
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(numcl, activation = 'softmax'))

    return model



model = buildMod(48, 48, 7)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 48, 48, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 48, 48, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 24, 24, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 24, 24, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 24, 24, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 24, 24, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 12, 12, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 12, 12, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 12, 12, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 12, 12, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 368,023 (1.40 MB)

 Trainable params: 368,023 (1.40 MB)

 Non-trainable params: 0 (0.00 B)

## Allenamento del modello

In [9]:
trDir = "Dataset/train"  
valDir = "Dataset/test"  
datagen = ImageDataGenerator(rescale=1./255)

# Dataset di training
trDat = datagen.flow_from_directory(
    trDir,
    target_size=(48, 48),
    batch_size=64,
    class_mode="categorical",
    shuffle=True
)

# Dataset di validation
valDat = datagen.flow_from_directory(
    valDir,
    target_size=(48, 48),
    batch_size=64,
    class_mode="categorical",
    shuffle=True
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [10]:
earlyS = EarlyStopping(monitor='val_loss', patience=5)
hist = model.fit(trDat, epochs=200, validation_data=valDat, callbacks=[earlyS], batch_size=32)

Epoch 1/200
449/449 ━━━━━━━━━━━━━━━━━━━━ 279s 608ms/step - accuracy: 0.2337 - loss: 1.8329 - val_accuracy: 0.2471 - val_loss: 1.7892
Epoch 2/200
449/449 ━━━━━━━━━━━━━━━━━━━━ 257s 572ms/step - accuracy: 0.2893 - loss: 1.7465 - val_accuracy: 0.4106 - val_loss: 1.5364
Epoch 3/200
449/449 ━━━━━━━━━━━━━━━━━━━━ 254s 566ms/step - accuracy: 0.3850 - loss: 1.5831 - val_accuracy: 0.4525 - val_loss: 1.4082
Epoch 4/200
449/449 ━━━━━━━━━━━━━━━━━━━━ 251s 558ms/step - accuracy: 0.4279 - loss: 1.4791 - val_accuracy: 0.4836 - val_loss: 1.3329
Epoch 5/200
449/449 ━━━━━━━━━━━━━━━━━━━━ 251s 560ms/step - accuracy: 0.4559 - loss: 1.4007 - val_accuracy: 0.4948 - val_loss: 1.3013
Epoch 6/200
449/449 ━━━━━━━━━━━━━━━━━━━━ 264s 565ms/step - accuracy: 0.4869 - loss: 1.3443 - val_accuracy: 0.5156 - val_loss: 1.2525
Epoch 7/200
449/449 ━━━━━━━━━━━━━━━━━━━━ 255s 568ms/step - accuracy: 0.4987 - loss: 1.3076 - val_accuracy: 0.5267 - val_loss: 1.2213
Epoch 8/200
449/449 ━━━━━━━━━━━━━━━━━━━━ 257s 572ms/step - accuracy: 

In [ ]:
model.save("Modelli/emo_v2.keras")